In [1]:
import pandas as pd
import numpy as np
import pyowm
import json
import pprint
import datetime
from datetime import timedelta

API_KEY = '2b54876f31212bf7fb13d0d0b62b911d'
owm = pyowm.OWM(API_KEY)
location_plant = {}
obj = datetime.datetime(2017, 4, 10).date()
location_plant[6] = [(1.696974, 41.844405), obj]

#location_plant[7] = [(30.778, 45.96), 1]
#location_plant[8] = [(30.578, 45.86), 2]

# Calculate reference evapotranspiration PART 1

Source:  http://pcse.readthedocs.io/en/master/_modules/pcse/util.html

Changed some things in the source code

In [2]:
# -*- coding: utf-8 -*-
# Copyright (c) 2004-2016 Alterra, Wageningen-UR
# Allard de Wit (allard.dewit@wur.nl), October 2016
"""Miscellaneous utilities for PCSE
"""
import os, sys
import copy
from math import log10, cos, sin, asin, sqrt, exp, log
from collections import namedtuple
from bisect import bisect_left
import textwrap
import sqlite3
import requests
import json
import weather


Celsius2Kelvin = lambda x: x + 273.16
hPa2kPa = lambda x: x/10

# Saturated Vapour pressure [kPa] at temperature temp [C]
SatVapourPressure = lambda temp: 0.6108 * exp((17.27 * temp) / (237.3 + temp))

temperature: 25.16
clouds: 40
rain: 0
altitude: 1019
Champaign


In [3]:
def penman_monteith(DAY, LAT, ELEV, TMIN, TMAX, AVRAD, VAP, WIND2):
    """Calculates reference ET0 based on the Penman-Monteith model.

     This routine calculates the potential evapotranspiration rate from
     a reference crop canopy (ET0) in mm/d. For these calculations the
     analysis by FAO is followed as laid down in the FAO publication
     `Guidelines for computing crop water requirements - FAO Irrigation
     and drainage paper 56 <http://www.fao.org/docrep/X0490E/x0490e00.htm#Contents>`_

    Input variables::

        DAY   -  Python datetime.date object                   -
        LAT   -  Latitude of the site                        degrees
        ELEV  - Elevation above sea level                      m
        TMIN  - Minimum temperature                            C
        TMAX  - Maximum temperature                            C
        AVRAD - Daily shortwave radiation                   J m-2 d-1
        VAP   - 24 hour average vapour pressure               hPa
        WIND2 - 24 hour average windspeed at 2 meter          m/s

    Output is:

        ET0   - Penman-Monteith potential transpiration
                rate from a crop canopy                     [mm/d]
    """

    # psychrometric instrument constant (kPa/Celsius)
    PSYCON = 0.665
    # albedo and surface resistance [sec/m] for the reference crop canopy
    REFCFC = 0.23; CRES = 70.
    # latent heat of evaporation of water [J/kg == J/mm] and
    LHVAP = 2.45E6
    # Stefan Boltzmann constant (J/m2/d/K4, e.g multiplied by 24*60*60)
    STBC = 4.903E-3
    # Soil heat flux [J/m2/day] explicitly set to zero
    G = 0.

    # mean daily temperature (Celsius)
    TMPA = (TMIN+TMAX)/2.

    # Vapour pressure to kPa
    VAP = hPa2kPa(VAP)

    # atmospheric pressure (kPa)
    T = Celsius2Kelvin(TMPA)
    PATM = 101.3 * pow((T - (0.0065*ELEV))/T, 5.26)

    # psychrometric constant (kPa/Celsius)
    GAMMA = PSYCON * PATM * 1.0E-3

    # Derivative of SVAP with respect to mean temperature, i.e.
    # slope of the SVAP-temperature curve (kPa/Celsius);
    SVAP_TMPA = SatVapourPressure(TMPA)
    DELTA = (4098. * SVAP_TMPA)/pow((TMPA + 237.3), 2)

    # Daily average saturated vapour pressure [kPa] from min/max temperature
    SVAP_TMAX = SatVapourPressure(TMAX)
    SVAP_TMIN = SatVapourPressure(TMIN)
    SVAP = (SVAP_TMAX + SVAP_TMIN) / 2.

    # measured vapour pressure not to exceed saturated vapour pressure
    VAP = min(VAP, SVAP)

    # Longwave radiation according at Tmax, Tmin (J/m2/d)
    # and preliminary net outgoing long-wave radiation (J/m2/d)
    STB_TMAX = STBC * pow(Celsius2Kelvin(TMAX), 4)
    STB_TMIN = STBC * pow(Celsius2Kelvin(TMIN), 4)
    RNL_TMP = ((STB_TMAX + STB_TMIN) / 2.) * (0.34 - 0.14 * sqrt(VAP))

    # Clear Sky radiation [J/m2/DAY] from Angot TOA radiation
    # the latter is found through a call to astro()
    r = astro(DAY, LAT, AVRAD)
    CSKYRAD = (0.75 + (2e-05 * ELEV)) * r.ANGOT

    if CSKYRAD > 0:
        # Final net outgoing longwave radiation [J/m2/day]
        RNL = RNL_TMP * (1.35 * (AVRAD/CSKYRAD) - 0.35)

        # radiative evaporation equivalent for the reference surface
        # [mm/DAY]
        RN = ((1-REFCFC) * AVRAD - RNL)/LHVAP

        # aerodynamic evaporation equivalent [mm/day]
        EA = ((900./(TMPA + 273)) * WIND2 * (SVAP - VAP))

        # Modified psychometric constant (gamma*)[kPa/C]
        MGAMMA = GAMMA * (1. + (CRES/208.*WIND2))

        # Reference ET in mm/day
        ET0 = (DELTA * (RN-G))/(DELTA + MGAMMA) + (GAMMA * EA)/(DELTA + MGAMMA)
        ET0 = max(0., ET0)
    else:
        ET0 = 0.

    return ET0

In [4]:
def wind10to2(wind10):
    """Converts windspeed at 10m to windspeed at 2m using log. wind profile
    """
    wind2 = wind10 * (log10(2./0.033) / log10(10/0.033))
    return wind2



def ea_from_tdew(tdew):
    """
    Calculates actual vapour pressure, ea [kPa] from the dewpoint temperature
    using equation (14) in the FAO paper. As the dewpoint temperature is the
    temperature to which air needs to be cooled to make it saturated, the
    actual vapour pressure is the saturation vapour pressure at the dewpoint
    temperature. This method is preferable to calculating vapour pressure from
    minimum temperature.

    Taken from fao_et0.py written by Mark Richards

    Reference:
    Allen, R.G., Pereira, L.S., Raes, D. and Smith, M. (1998) Crop
        evapotranspiration. Guidelines for computing crop water requirements,
        FAO irrigation and drainage paper 56)

    Arguments:
    tdew - dewpoint temperature [deg C]
    """
    # Raise exception:
    if (tdew < -95.0 or tdew > 65.0):
        # Are these reasonable bounds?
        msg = 'tdew=%g is not in range -95 to +60 deg C' % tdew
        raise ValueError(msg)

    tmp = (17.27 * tdew) / (tdew + 237.3)
    ea = 0.6108 * exp(tmp)
    return ea


#edit: changed default values of cA and cB, FAO recommends these values if calibrated values are not available
def angstrom(day, latitude, ssd, cA=0.5, cB=0.25):
    """Compute global radiation using the Angstrom equation.
    
    Global radiation is derived from sunshine duration using the Angstrom
    equation:
    globrad = Angot * (cA + cB * (sunshine / daylength)
    
    :param day: day of observation (date object)
    :param latitude: Latitude of the observation
    :param ssd: Observed sunshine duration
    :param cA: Angstrom A parameter
    :param cB: Angstrom B parameter    
    :returns: the global radiation in J/m2/day
    """
    r = astro(day, latitude, 0)
    globrad = r.ANGOT * (cA + cB * (ssd / r.DAYL))
    return globrad

                       

def doy(day):
    """Converts a date or datetime object to day-of-year (Jan 1st = doy 1)
    """
    # Check if day is a date or datetime object
    if isinstance(day, (datetime.date, datetime.datetime)):
        return day.timetuple().tm_yday
    else:
        msg = "Parameter day is not a date or datetime object."
        raise RuntimeError(msg)

In [5]:
def astro(day, latitude, radiation, _cache={}):
    """python version of ASTRO routine by Daniel van Kraalingen.
    
    This subroutine calculates astronomic daylength, diurnal radiation
    characteristics such as the atmospheric transmission, diffuse radiation etc.

    :param day:         date/datetime object
    :param latitude:    latitude of location
    :param radiation:   daily global incoming radiation (J/m2/day)

    output is a `namedtuple` in the following order and tags::

        DAYL      Astronomical daylength (base = 0 degrees)     h      
        DAYLP     Astronomical daylength (base =-4 degrees)     h      
        SINLD     Seasonal offset of sine of solar height       -      
        COSLD     Amplitude of sine of solar height             -      
        DIFPP     Diffuse irradiation perpendicular to
                  direction of light                         J m-2 s-1 
        ATMTR     Daily atmospheric transmission                -      
        DSINBE    Daily total of effective solar height         s
        ANGOT     Angot radiation at top of atmosphere       J m-2 d-1
 
    Authors: Daniel van Kraalingen
    Date   : April 1991
 
    Python version
    Author      : Allard de Wit
    Date        : January 2011
    """

    # Check for range of latitude
    if abs(latitude) > 90.:
        msg = "Latitude not between -90 and 90"
        raise RuntimeError(msg)
    LAT = latitude
        
    # Determine day-of-year (IDAY) from day
    IDAY = doy(day)
    
    # reassign radiation
    AVRAD = radiation

    # Test if variables for given (day, latitude, radiation) were already calculated
    # in a previous run. If not (e.g. KeyError) calculate the variables, store
    # in cache and return the value.
    try:
        return _cache[(IDAY, LAT, AVRAD)]
    except KeyError:
        pass

    # constants
    RAD = 0.0174533
    PI = 3.1415926
    ANGLE = -4.

    # map python functions to capitals
    SIN = sin
    COS = cos
    ASIN = asin
    REAL = float
    SQRT = sqrt
    ABS = abs

    # Declination and solar constant for this day
    DEC = -ASIN(SIN(23.45*RAD)*COS(2.*PI*(REAL(IDAY)+10.)/365.))
    SC  = 1370.*(1.+0.033*COS(2.*PI*REAL(IDAY)/365.))

    # calculation of daylength from intermediate variables
    # SINLD, COSLD and AOB
    SINLD = SIN(RAD*LAT)*SIN(DEC)
    COSLD = COS(RAD*LAT)*COS(DEC)
    AOB = SINLD/COSLD

    # For very high latitudes and days in summer and winter a limit is
    # inserted to avoid math errors when daylength reaches 24 hours in 
    # summer or 0 hours in winter.

    # Calculate solution for base=0 degrees
    if abs(AOB) <= 1.0:
        DAYL  = 12.0*(1.+2.*ASIN(AOB)/PI)
        # integrals of sine of solar height
        DSINB  = 3600.*(DAYL*SINLD+24.*COSLD*SQRT(1.-AOB**2)/PI)
        DSINBE = 3600.*(DAYL*(SINLD+0.4*(SINLD**2+COSLD**2*0.5))+
                 12.*COSLD*(2.+3.*0.4*SINLD)*SQRT(1.-AOB**2)/PI)
    else:
        if AOB >  1.0: DAYL = 24.0
        if AOB < -1.0: DAYL = 0.0
        # integrals of sine of solar height	
        DSINB = 3600.*(DAYL*SINLD)
        DSINBE = 3600.*(DAYL*(SINLD+0.4*(SINLD**2+COSLD**2*0.5)))

    # Calculate solution for base=-4 (ANGLE) degrees
    AOB_CORR = (-SIN(ANGLE*RAD)+SINLD)/COSLD
    if abs(AOB_CORR) <= 1.0:
        DAYLP = 12.0*(1.+2.*ASIN(AOB_CORR)/PI)
    elif AOB_CORR > 1.0:
        DAYLP = 24.0
    elif AOB_CORR < -1.0:
        DAYLP = 0.0

    # extraterrestrial radiation and atmospheric transmission
    ANGOT = SC*DSINB
    # Check for DAYL=0 as in that case the angot radiation is 0 as well
    if DAYL > 0.0:
        ATMTR = AVRAD/ANGOT
    else:
        ATMTR = 0.

    # estimate fraction diffuse irradiation
    if (ATMTR > 0.75):
        FRDIF = 0.23
    elif (ATMTR <= 0.75) and (ATMTR > 0.35):
        FRDIF = 1.33-1.46*ATMTR
    elif (ATMTR <= 0.35) and (ATMTR > 0.07):
        FRDIF = 1.-2.3*(ATMTR-0.07)**2
    else:  # ATMTR <= 0.07
        FRDIF = 1.

    DIFPP = FRDIF*ATMTR*0.5*SC
    
    # Pack return values in namedtuple, add to cache and return
    astro_nt = namedtuple("AstroResults","DAYL, DAYLP, SINLD, COSLD, DIFPP, "
                                         "ATMTR, DSINBE, ANGOT")
    retvalue = astro_nt(DAYL, DAYLP, SINLD, COSLD, DIFPP, ATMTR, DSINBE, ANGOT)
    _cache[(IDAY, LAT, AVRAD)] = retvalue

    return retvalue

In [6]:
def get_elevation(coords):
    """
    Compute elevation of a geographical location whose coordinates are given
    This uses the google map API
    
    @param: a tuple of geographical coordinates; (latitude, longitude)
    @return: elevation in meters
    """

    r = requests.get('http://maps.googleapis.com/maps/api/elevation/json?locations='+str(coords[0])+','+str(coords[1])+'&sensor=true_or_false')
    #elevation = r['results'][0]['elevation']
    if r.status_code == 200:
        parsed = json.loads(r.content)
        return parsed['results'][0]['elevation']
    else:
        msg = "Bad response from google"
        raise RuntimeError(msg)

In [7]:
def get_dewpoint(temperature, relative_humidity):
    #source - https://iridl.ldeo.columbia.edu/dochelp/QA/Basic/dewpoint.html
    """
    Calculate the dew point from current temperature and relative humidity.
    Dew point is then used to calculate vapor pressure.
    @param - temperature (in C)
    @param - relative_humidity (%age)
    @return - dew point (in C)
    
    Constants used:
    E0 = 0.611 kPa
    T0 = 273.16 K
    L_RV = 5423 K
    """
    
    E0 = 0.611
    L_RV = 5423
    T0 = 273.16
    T = Celsius2Kelvin(temperature)
    E_S = E0 * exp(L_RV * ((1/T0)-(1/T)))
    E = (relative_humidity/100)*E_S
    T_D = 1/(1/T0 - (log(E/E0)*(1/L_RV)))
    return T_D - 273.16

In [8]:
def get_min_max_temp(COORDS, API_K):
    """
    Gets the minimum and maximum temperature of the day of the coordinates given
    
    @param - COORDS ((latitude, longitude)); Coordinate of the point
    @param - API_K (API Key to use open weather map api)
    
    @return - Returns the minimum and maximum temperature as floats (min, max)
    """
    
    #get an owm instance using the key from home.openweathermap.org
    owm = pyowm.OWM(API_K)

    #get current weather
    observation = owm.weather_at_coords(COORDS[0], COORDS[1])
    weather = observation.get_weather()
    temp = weather.get_temperature('celsius')
    
    return (temp['temp_min'], temp['temp_max'])

def get_current_temp(COORDS, API_K):
    """
    Gets the current temperature of the coordinates given
    
    @param - COORDS ((latitude, longitude)); Coordinate of the point
    @param - API_K (API Key to use open weather map api)
    
    @return - Returns the current temperature as a float
    """
    owm = pyowm.OWM(API_K)

    #get current weather
    observation = owm.weather_at_coords(COORDS[0], COORDS[1])
    weather = observation.get_weather()
    temp = weather.get_temperature('celsius')

    return temp['temp']

def get_current_humidity(COORDS, API_K):
    """
    Gets the current humidity of the coordinates given
    
    @param - COORDS ((latitude, longitude)); Coordinate of the point
    @param - API_K (API Key to use open weather map api)
    
    @return - Returns the current humidity as %
    """
    owm = pyowm.OWM(API_K)

    #get current weather
    observation = owm.weather_at_coords(COORDS[0], COORDS[1])
    weather = observation.get_weather()
    humid = weather.get_humidity()
    
    return humid

def get_current_wind(COORDS, API_K):
    """
    Gets the current wind speed of the coordinates given
    
    @param - COORDS ((latitude, longitude)); Coordinate of the point
    @param - API_K (API Key to use open weather map api)
    
    @return - Returns the current wind speed (m/s)
    """
    owm = pyowm.OWM(API_K)
    
    observation = owm.weather_at_coords(COORDS[0], COORDS[1])
    weather = observation.get_weather()
    wind = weather.get_wind()
    
    return wind['speed']

In [9]:
# The 6 dictionaries below store the length of plant growth in different regions and plant months
cent_india = {'ini' : 15, 'dev' : 25, 'mid' : 50, 'late' : 30, 'total' : 120}
t30_45L = {'ini' : 20, 'dev' : 25, 'mid' : 60, 'late' : 30, 'total' : 135}
east_africa = {'ini' : 15, 'dev' : 30, 'mid' : 65, 'late' : 40, 'total' : 150}
april = {'ini' : 40, 'dev' : 30, 'mid' : 40, 'late' : 20, 'total' : 130}
nov = {'ini' : 40, 'dev' : 60, 'mid' : 60, 'late' : 40, 'total' : 200}
cali_usa = {'ini' : 20, 'dev' : 50, 'mid' : 60, 'late' : 30, 'total' : 160}

# Central coordinate of different regions of plantation
all_coords = {'cent_india' : (21.115733, 79.099871), 'east_africa' : (3.116045, 35.604941), 'cali_usa' : (34.995219, -116.126399)}

# Radii from central coordi
radii = {'cent_india' : 572, 'east_africa' : 1226, 'cali_usa' : 181}

kc = {'ini' : 0.3, 'mid' : 1.15, 'end' : 0.25}
lengths = {'cent_india' : cent_india, 't30_45L' : t30_45L, 'east_africa' : east_africa, 'april' : april, 'nov' : nov, 'cali_usa' : cali_usa}

from geopy.distance import vincenty

def get_distance(coordA, coordB):
    """
    Calculates the distance between two coordinates in the world
    
    @param - coordA ((latitude, longitude)); Coordinates of point A
    @param - coordB ((latitude, longitude)); Coordinates of point B
    
    @return - Distance between two coordinates in kilometers
    """
    return vincenty(coordA, coordB).kilometers

def get_region(coords):
    """
    Gets the region of the coordinates of the plant (California, Central India, East Africa or None)
    
    @param - coords ((latitude, longitude))
    
    @return - Region of plant in the world (string)
    """
    for key, value in all_coords.items():
        if (get_distance(value, coords) < radii[key]):
            return key
    return None

def get_kc_mid(wind_speed, rel_humid, height):
    """
    Calculates Kc value in the middle period of plant growth based on the following parameters.
    
    @param - wind_speed (m/s)
    @param - rel_humid (%)
    @param - height (m)
    
    @return - Kc value in the middle period of plant growth (float)
    """
    kc_mid = kc['mid'] + 0.04 * (wind_speed - 2) - 0.004 * (rel_humid - 45) * ((height / 3)**3)
    return kc_mid

def get_kc_end(wind_speed, rel_humid, height):
    """
    Calculates Kc value in the end period of plant growth based on the following parameters.
    
    @param - wind_speed (m/s)
    @param - rel_humid (%)
    @param - height (m)
    
    @return Kc value in the end period of plant growth
    """
    kc_end = kc['end'] + 0.04 * (wind_speed - 2) - 0.004 * (rel_humid - 45) * ((height / 3)**3)
    return kc_end

def get_kc_value(plant_date, day, coords, wind_speed, rel_humid, height):
    """
    Calculates the Kc Value from the parameters. This Kc value is then used to obtain crop evapotranspiration
    which is a product of Kc and ET0.
    
    @param - plant_date (in datetime.date object)
    @param - day (integer); Number of days since plant date
    @param - coords ((latitude, longitude))
    @param - wind_speed (m/s)
    @param - rel_humid (%)
    @param - height (m)

    @return Kc value of plant
    """
    
    region = get_region(coords)
    
    if (region == None):
        month = plant_date.month
        
        if (month == 4):
            region = 'april'
        if (month == 11):
            region = 'nov'
    
    length = lengths[region]
    
    if day > length['total'] or day < 0:
        return 'invalid'
    if day <= length['ini']:
        return 'dummy value'
    if day <= length['mid']:
        return get_kc_mid(wind_speed, rel_humid, height)
    
    return get_kc_end(wind_speed, rel_humid, height)

In [10]:
def calc_etc(location, plant_date):
    
    """
        DAY   -  Python datetime.date object                   -
        LAT   -  Latitude of the site                        degrees
        ELEV  - Elevation above sea level                      m
        TMIN  - Minimum temperature                            C
        TMAX  - Maximum temperature                            C
        AVRAD - Daily shortwave radiation                   J m-2 d-1
        VAP   - 24 hour average vapour pressure               hPa
        WIND2 - 24 hour average windspeed at 2 meter          m/s 
        
    """
    
    date = datetime.datetime.now().date()
    lat = location[0]
    elev = get_elevation(location)
    temps = get_min_max_temp(location, API_KEY)
    tmin = temps[0]
    tmax = temps[1]
    avrad = angstrom(date, lat, 7)
    vap = ea_from_tdew(get_dewpoint(get_current_temp(location, API_KEY), get_current_humidity(location, API_KEY)))
    wind = wind10to2(get_current_wind(location, API_KEY)) 
    
    eto = penman_monteith(date, lat, elev, tmin, tmax, avrad, vap, wind)
    
    diff_days = (date - plant_date).days
    height = 5
    kc_val = get_kc_value(plant_date, diff_days, location, wind, get_current_humidity(location, API_KEY), height)
    
    #print (diff_days)
    #print (wind)
    #print (get_current_humidity(location, API_KEY))
    print (eto)
    print (kc_val)
    if (isinstance(kc_val, str)):
        return None
    etc = eto * kc_val
    return etc

In [12]:
obj = datetime.datetime(2017, 4, 10).date()
print (calc_etc((1.696974, 41.844405), obj))

8.876315825098256
-0.21388992523142258
-1.8985545281607592


# PART 2 Get Data

In [13]:
#open JSON file
data = open('budlabsensor-sensor6-export.json').read()
df = pd.read_json(data)
df.head()

,2017-08-24 13:45:39,2017-08-24 13:46:14,2017-08-24 13:46:17,2017-08-24 13:52:57,2017-08-24 13:54:09,2017-08-24 13:56:00,2017-08-24 13:56:29
color_blue,78,76,76,87,83,85,86
color_green,84,81,81,88,85,85,86
color_red,115,118,119,101,99,96,92
cr,5,5,5,5,5,5,5
datarate,SF12BW125,SF12BW125,SF12BW125,SF12BW125,SF12BW125,SF12BW125,SF12BW125


In [14]:
df = df.transpose()
df.head()

,color_blue,color_green,color_red,cr,datarate,gateway_eui,humidity,len,node_eui,rssi,seq,snr,temperature,time,type
2017-08-24 13:45:39,78,84,115,5,SF12BW125,00000027EB89E1CA,53.10,33,6,-55,0,8,23.40,2017-08-24T13:45:39.279110,24
2017-08-24 13:46:14,76,81,118,5,SF12BW125,00000027EB89E1CA,53.00,33,6,-58,0,6,23.40,2017-08-24T13:46:14.911401,24
2017-08-24 13:46:17,76,81,119,5,SF12BW125,00000027EB89E1CA,53.00,33,6,-56,0,6,23.40,2017-08-24T13:46:17.976941,24
2017-08-24 13:52:57,87,88,101,5,SF12BW125,00000027EB89E1CA,53.50,33,6,-60,0,6,25.10,2017-08-24T13:52:57.556793,24
2017-08-24 13:54:09,83,85,99,5,SF12BW125,00000027EB89E1CA,50.30,32,6,-60,0,6,25.80,2017-08-24T13:54:09.970341,24


In [15]:
df = df[['color_blue', 'color_green', 'color_red', 
         'datarate', 'humidity', 'node_eui', 'rssi', 
         'snr' , 'temperature', 'time']]
df.head()

,color_blue,color_green,color_red,datarate,humidity,node_eui,rssi,snr,temperature,time
2017-08-24 13:45:39,78,84,115,SF12BW125,53.10,6,-55,8,23.40,2017-08-24T13:45:39.279110
2017-08-24 13:46:14,76,81,118,SF12BW125,53.00,6,-58,6,23.40,2017-08-24T13:46:14.911401
2017-08-24 13:46:17,76,81,119,SF12BW125,53.00,6,-56,6,23.40,2017-08-24T13:46:17.976941
2017-08-24 13:52:57,87,88,101,SF12BW125,53.50,6,-60,6,25.10,2017-08-24T13:52:57.556793
2017-08-24 13:54:09,83,85,99,SF12BW125,50.30,6,-60,6,25.80,2017-08-24T13:54:09.970341


In [16]:
print(df)

                    color_blue color_green color_red   datarate humidity  \
2017-08-24 13:45:39         78          84       115  SF12BW125    53.10   
2017-08-24 13:46:14         76          81       118  SF12BW125    53.00   
2017-08-24 13:46:17         76          81       119  SF12BW125    53.00   
2017-08-24 13:52:57         87          88       101  SF12BW125    53.50   
2017-08-24 13:54:09         83          85        99  SF12BW125    50.30   
2017-08-24 13:56:00         85          85        96  SF12BW125    49.90   
2017-08-24 13:56:29         86          86        92  SF12BW125    49.70   

                    node_eui rssi snr temperature                        time  
2017-08-24 13:45:39        6  -55   8       23.40  2017-08-24T13:45:39.279110  
2017-08-24 13:46:14        6  -58   6       23.40  2017-08-24T13:46:14.911401  
2017-08-24 13:46:17        6  -56   6       23.40  2017-08-24T13:46:17.976941  
2017-08-24 13:52:57        6  -60   6       25.10  2017-08-24T13:52:57.

# PART 3 Calculations from data

We create the following dataframe (called monitor) with these columns:
1. Date
2. Hour
3. Sunrise
4. Sunset
5. Atmospheric Temperature
6. Atmospheric Humidity
7. Clouds
8. Rain
9. Windspeed
10. Sensor temperature
11. sensor humidity
12. sensor R
13. sensor G
14. sensor B


In [17]:
cols = ['date', 'hour', 'sunrise','sunset','temperature_a','temp_min_a','temp_max_a','humidity_a', 'clouds','rain','windspeed', 'temperature_c', 'humidity_c','r','g','b', 'gdd', 'etc']
monitor = {}
for key in location_plant:
    monitor[key] = pd.DataFrame()

for key in monitor:
    for col in cols:
        monitor[key][col] = None

In [18]:
def parseDate(dateStr, Format):
    
    # function to convert str Date to Datetime type
    # dateStr: String of Date; Format: String representing date format
    # output: Datetime object
    
    if pd.isnull(dateStr):
        return np.nan
    return datetime.datetime.strptime(dateStr, Format)

In [19]:
def get_gdd(cur_gdd, min_temp, max_temp):
    if min_temp < 0:
        min_temp = 0
    if cur_gdd <384:
            if max_temp> 21:
                max_temp = 21
            elif max_temp > 35:
                max_temp = 35

    return (cur_gdd + (min_temp+max_temp)/2) 

In [20]:
def update_monitors(monitor, df):
    """
    @param: monitor; dataframe that stores all the conditions for all the locations for past 14 days
    @param: df; dataframe with the information from firebase database, all data from sensors
    """
    cur_time = datetime.datetime.now()
    
    for key in location_plant:
        temp_df = {}
        print(key)
        new_df = df[df['node_eui'] == key]
        print(new_df)
        observation = owm.weather_at_coords(location_plant[key][0][0], location_plant[key][0][1])
        weather = observation.get_weather()
        temp_df['date'] = cur_time.day
        temp_df['hour'] = cur_time.hour
        sunrise =  weather.get_sunrise_time('iso')[0:-3]
        sunrise = parseDate(sunrise, "%Y-%m-%d %H:%M:%S")
        sunset = weather.get_sunrise_time('iso')[0:-3]
        sunset = parseDate(sunset, "%Y-%m-%d %H:%M:%S")
        temp_df['sunrise'] = sunrise
        temp_df['sunset'] = sunset
        temp_df['temperature_a'] = weather.get_temperature('celsius')['temp']
        temp_df['temp_min_a'] = weather.get_temperature('celsius')['temp_min']
        temp_df['temp_max_a'] = weather.get_temperature('celsius')['temp_max']
        temp_df['humidity_a'] = weather.get_humidity()
        temp_df['clouds'] = weather.get_clouds()
        temp_df['etc'] = calc_etc(location_plant[key][0], location_plant[key][1])
        rain = weather.get_rain()
        if len(rain)==0:
            rain = 0
        else:
            rain = rain['3h']
        temp_df['rain'] = rain
        temp_df['windspeed'] = weather.get_wind()['speed']
        temp_df['temperature_c'] = new_df['temperature'][-1]
        temp_df['humidity_c'] = new_df['humidity'][-1]
        temp_df['r'] = new_df['color_red'][-1]
        temp_df['g'] = new_df['color_green'][-1]
        temp_df['b'] = new_df['color_blue'][-1]
        
        if (cur_time.day == location_plant[key][1].day):
            temp_df['gdd'] = 0
        elif (len(monitor[key]) <= 1):
            temp_df['gdd'] = 0
        else:
            temp_df['gdd'] = monitor[key]['gdd'][-1]
            
        if (cur_time.hour == 12 or cur_time.hour == 13):
            gdd_1 = monitor[key]['gdd'][-1]
            gdd_2 = monitor[key]['gdd'][-2]
            if (gdd_1 == gdd_2):
                temp_df['gdd'] = get_gdd(temp_df['gdd'], temp_df['temp_min_a'], temp_df['temp_max_a'])
        print(temp_df)
        monitor[key] = monitor[key].append(temp_df, ignore_index=True)
        print(monitor[key]['gdd'])
        
        #if monitor[key].shape[0] > 350:
            #monitor[key].drop(monitor[key].index(0), in_place = True)
        #monitor[key].reset_index(inplace=True, drop=True)

In [21]:
def has_barley_stripe(monitor, key):
    if (len(monitor[key]) < 16):
        return False
    
    last_16 = monitor[key].tail(n=16)
    cur_gdd = monitor[key].loc(monitor[key].shape[0]-1)
    if (cur_gdd > 1300):
        optimum_temp_len = len(last_16[10 <= last_16['temperature_a'] <= 14])
        if optimum_temp_len == 16:
            return True
    
    return False
    
        

In [22]:
def has_FHB(monitor, key):
    if (len(monitor[key]) < 72):
        return False
    
    last_3_days = monitor[key].tail(n=72)
    cur_gdd = last_3_days.loc[last_3_days.shape[0]-1]['gdd']
    if (cur_gdd > 1200):
        optimum_temp_len = len(last_3_days[22 <= last_3_days['temperature_a'] <= 32])
        optimum_moist_len = len(last_3_days[last_3_days['humidity_c'] >= 70])
        if (optimum_moist_len > 65 and optimum_temp_len > 65):
            return True
        
    return False

In [23]:
def has_scald(monitor, key):
    if (len(monitor[key]) < 72):
        return False
    
    last_72 = monitor[key].tail(n = 72)
    
    # Spore production optimal at temperatures 7.5 - 12.5 after 72 hrs of wetting
    avg_temp_last_72 = last_72['temperature_a'].sum() / 72
    if (last_72['rain'].sum() / 216 > 1 and avg_temp_last_72 >= 7.5 and avg_temp_last_72 <= 12.5):
        return True
    
    last_24 = monitor[key].tail(n = 24)

    # After 24 hrs of dark/moist, spores can infect (moist = humid > 65)
    num_clouds = len(last_24[last_24['clouds'] > 85])
    num_humid = len(last_24[last_24['humidity_a'] > 65])
    if (num_humid >= 22 and num_clouds >= 22):
        return True
    
    last_48 = monitor[key].tail(n = 48)
    
    # If wet (48 hrs) + 10-18 C, conidia forms
    num_temp = len(last_48[last_48['temperature_a'] >= 10 and last_48['temperature_a'] <= 18])
    num_rain = last_48['rain'].sum() / 144
    if (num_temp >= 22 and num_rain >= 1):
        return True
    
    return False

In [24]:
def has_covered_smuts(monitor, key):
    if (len(monitor[key]) < 24):
        return False
    
    last_24 = monitor[key].tail(n = 24)
    
    # Infection occurs at temperatures 20 - 25 after wetting
    num_temp = len(last_24[last_24['temperature_a'] >= 20 and last_24['temperature_a'] <= 25])
    num_rain = last_24['rain'].sum() / 72
    if ((500 >= last_24.loc[len(last_24) - 1]['gdd'] >= 235) and num_temp >= 22 and num_rain >= 1):
        return True
    
    return False

In [25]:
def has_loose_smut(monitor, key):
    if (len(monitor[key]) < 24):
        return False
    last_24 = monitor[key].tail(n = 24)
    
    num_temp = len(last_24[last_24['temperature_a'] >= 16 and last_24['temperature_a'] <= 22])
    num_clouds = len(last_24[last_24['clouds'] > 85])
    if (num_temp >= 22 and num_clouds >= 22):
        return True
    
    return False

In [26]:
def has_net_blotch (monitor, key):
    if (len(monitor[key]) < 24):
        return False
    df = monitor[key] #get DataFrame from correct sensor
    nb_gdd = True
    gdd = df.tail(n=1)['gdd'] #gets gdd in last row of df (most recent gdd)
    nb_gdd = (gdd > 430) #first condition: is gdd greater than 430
    
    nb_temp = True
    temp = df.tail(n=24)['temperature_a'] #gets last 24 hrs worth of atmospheric temperature readings
    #loop runs through all temps for last day and if any are below 15 or above 25, condition is not met
    for t in temp:
        if (t<15 or t>25):
            nb_temp = False
            break
    
    nb_humid = True
    humid = df.tail(n=24)['humidity_a'] #gets last 24 hrs worth of atmospheric humidity readings
    #loop runs through all temps for last day and if any are below 90 (%), condition is not met
    for h in humid:
        if (h<90):
            nb_humid = False
    
    nb_clouds = True
    clouds = df.tail(n=24)['clouds'] #gets last 24 hrs worth of % of clouds in the sky
    #loop runs through all temps for last day and if any are below 15 or above 25, condition is not met
    for c in clouds:
        if (c<30):
            nb_clouds = False
    
    conditions = [nb_gdd, nb_temp, nb_humid, nb_clouds]
    #check if any conditions are not met. if not met, disease is not there, so return false
    for c in conditions:
        if (not c):
            return False
    return True

In [27]:
def has_spot_blotch (monitor, key):
    """
    Longer than 16 hrs above 20 C / moist weather is necessary
    spot blotch can occur at any stage of plant development (no need to check gdd)
    """
    if (len(monitor[key]) < 16):
        return False
    
    df = monitor[key]
    temp = df.tail(n=16) #last 16 hrs of temp readings
    humid = df.tail(n=16) #last 16 hrs of humidity readings
    #loop checks if temp or humidity dipped below required amount in last 16 hrs
    for i in range (0,16):
        if ( (temp[i] < 20) or (humid[i] < 65) ):
            return False
    return True
    

In [28]:
def has_leaf_rust (monitor, key):
    """
    Leaf rust often is just present because of previous crops, so this function will predict leaf rust spreading
    (as opposed to predicting the disease occuring itself)
    """
    if (len(monitor[key]) < 72):
        return False
    #leaf rust requires 15-22 C and some amount of wetness (rainfall) for epidemic development
    df = monitor[key]
    lr_temp = True
    temp = df.tail(n=72) #last 72 hrs of temp readings
    for t in temp:
        if (t<15 or t>22):
            lr_temp = False
    
    lr_rain = True
    rain = df.tail(n=72) #last 72 hrs of rain readings
    count_rain = 0
    for r in rain:
        if (r>0):
            count_rain += 1
    if (count_rain <= 36):
        lr_rain = False
    
    if (lr_rain and lr_temp):
        return True
    
    return False

In [29]:
update_monitors(monitor, df)

6
                    color_blue color_green color_red   datarate humidity  \
2017-08-24 13:45:39         78          84       115  SF12BW125    53.10   
2017-08-24 13:46:14         76          81       118  SF12BW125    53.00   
2017-08-24 13:46:17         76          81       119  SF12BW125    53.00   
2017-08-24 13:52:57         87          88       101  SF12BW125    53.50   
2017-08-24 13:54:09         83          85        99  SF12BW125    50.30   
2017-08-24 13:56:00         85          85        96  SF12BW125    49.90   
2017-08-24 13:56:29         86          86        92  SF12BW125    49.70   

                    node_eui rssi snr temperature                        time  
2017-08-24 13:45:39        6  -55   8       23.40  2017-08-24T13:45:39.279110  
2017-08-24 13:46:14        6  -58   6       23.40  2017-08-24T13:46:14.911401  
2017-08-24 13:46:17        6  -56   6       23.40  2017-08-24T13:46:17.976941  
2017-08-24 13:52:57        6  -60   6       25.10  2017-08-24T13:52:5

In [30]:
print(monitor[6])

  date hour             sunrise              sunset  temperature_a  \
0   24   15 2017-08-24 03:10:08 2017-08-24 03:10:08          25.46   

   temp_min_a  temp_max_a humidity_a clouds rain  windspeed temperature_c  \
0       25.46       25.46         74     64    0       5.33         26.60   

  humidity_c   r   g   b gdd       etc  
0      49.70  92  86  86   0 -1.898555  


In [31]:
import matplotlib.pyplot as plt
def show_graph(df2, attr):
    plt.xlabel('time')
    
    if (attr == 'temperature_c'):
        ylabel = 'Celsius'
    elif (attr == 'humidity_c'):
        ylabel = 'Percentage'
    elif (attr == 'etc'):
        ylabel = 'etc value'
    else:
        ylabel = 'color value'
        
    time = df2['date'] + df2['hour']
    
    print (df2[attr])
    plt.ylabel(ylabel)
    plt.plot([0, 10, 20], df2[attr].tolist(), 'ro')
    plt.axis([0, 100, 0, 100])
    plt.show()

In [32]:
def make_graphs(monitor, key):
    show_graph(monitor[key], 'temperature_c')
    show_graph(monitor[key], 'humidity_c')
    show_graph(monitor[key], 'r')
    show_graph(monitor[key], 'g')
    show_graph(monitor[key], 'b')
    show_graph(monitor[key], 'etc')

In [34]:
for key in location_plant:
    if (has_barley_stripe(monitor, key)):
        alert = "Barley stripe near "+key
        print(alert)
    else:
        print("No disease")
    
    if (has_FHB(monitor, key)):
        alert = "FHB near "+key
        print(alert)
    else:
        print("No disease")
    
    if (has_scald(monitor, key)):
        alert = "Scald near "+key
        print(alert)
    else:
        print("No disease")
    
    if (has_covered_smuts(monitor, key)):
        alert = "Covered smut near "+key
        print(alert)
    else:
        print("No disease")
        
    if (has_loose_smut(monitor, key)):
        alert = "Loose smut near "+key
        print(alert)
    else:
        print("No disease")
        
    if (has_net_blotch(monitor, key)):
        alert = "Net Blotch near "+key
        print(alert)
    else:
        print("No disease")
    
    if (has_spot_blotch(monitor, key)):
        alert = "Spot Blotch near "+key
        print(alert)
    else:
        print("No disease")
    
    if (has_leaf_rust(monitor, key)):
        alert = "Leaf rust near "+key
        print(alert)
    else:
        print("No disease")
    

No disease
No disease
No disease
No disease
No disease
No disease
No disease
No disease
